In [1]:
import os
import sys, glob
import argparse
# 異なる階層のutilsをインポートするために必要
sys.path.append("../")
import numpy as np
import math
import re
import random
from collections import defaultdict
# 異なる階層のutilsからインポート
from utils.constant import *
from utils.help_func import load_pickle, save_pickle, load_json, save_json
# 異なる階層のmodel_utilsからインポート
from model_utils.model_util import load_model, sent2tensor
from model_utils import train_args
from utils.constant import *
from utils.stat_util import *
from utils.help_func import load_pickle
from scipy import stats
import pandas as pd

In [2]:
B = 10
variants = ['srnn', 'gru', 'lstm']
thetas = [10, 50]
method_names = ['ochiai', 'tarantula', 'dstar', 'ochiai2', 'ample']
num_dataset = 8

# methodごとに検定結果をまとめる

In [3]:
res_arr = np.zeros((len(method_names), B, 3*2*8, 6*2))
stat_res_arr = np.full((len(method_names), 3*2*8, 6), False)
for mi, method in enumerate(method_names):
  print(f'------method={method}------')
  for bi in range(B):
    pafl_path = f'./artifacts/pafl_res_fortest/boot_{bi}/{method}_pafl_res.csv'
    rnnrepair_path = f'./artifacts/rnnrepair_res_fortest/boot_{bi}_res.csv'
    pafl_res = np.genfromtxt(pafl_path, delimiter=',', dtype='float').astype(np.float32)
    rnnrepair_res = np.genfromtxt(rnnrepair_path, delimiter=',', dtype='float').astype(np.float32)
    res_arr[mi][bi][:, :6] = pafl_res
    res_arr[mi][bi][:, 6:] = rnnrepair_res  
  cnt, err_cnt = 0, 0
  for i, elei in enumerate(stat_res_arr[mi]):
    for j, _ in enumerate(elei):
      pafl_group = res_arr[mi, :, i, j]
      rnnrepair_group = res_arr[mi, :, i, j+6]
      try:
        p_value = stats.mannwhitneyu(pafl_group, rnnrepair_group, alternative="less").pvalue
        p_value = round(p_value, 4)
      except ValueError as e:
        # print(e)
        # print(pafl_group, rnnrepair_group)
        err_cnt += 1
        continue
      u = stats.mannwhitneyu(pafl_group, rnnrepair_group, alternative="less").statistic
      d_cliff = - (2*u/(len(pafl_group)*len(rnnrepair_group)) - 1)
      d_cliff = round(d_cliff, 4)
      if np.mean(pafl_group) >= np.mean(rnnrepair_group):
        if d_cliff > 0:
          print(d_cliff)
      # p<0.01かつd_cliff>0474のを記憶
      if p_value < 0.01 and d_cliff > 0.474:
        cnt += 1
        stat_res_arr[mi][i][j] = True
  # print(err_cnt, 3*2*8*6*2) # 128 576
  print(cnt, 3*2*8*6, cnt/(3*2*8*6))
  # print(np.count_nonzero(stat_res_arr[mi]))
  print(cnt, 3*2*8*6*2-err_cnt)

------method=ochiai------
0.06
0.1
0.02
187 288 0.6493055555555556
187 560
------method=tarantula------
0.13
0.15
198 288 0.6875
198 559
------method=dstar------
0.07
0.06
0.02
178 288 0.6180555555555556
178 560
------method=ochiai2------
0.1
0.07
213 288 0.7395833333333334
213 560
------method=ample------
0.06
0.06
185 288 0.6423611111111112
185 559


# dataset, variantごとに検定結果をまとめる

In [4]:
print(res_arr.shape) # (method, boot, theta*variant*dataset, metrics*2)
print(stat_res_arr.shape) # (method, theta*variant*dataset, metrics)

(5, 10, 48, 12)
(5, 48, 6)


データセットごとに見るには，

stat_res_arrを `(dataset, theta*variant*method, metrics)` にかえればいい

In [17]:
deno = len(thetas) * 6 * len(method_names)
datasets = ['3', '4', '7', 'bp', 'mr', 'imdb', 'toxic', 'mnist']
dic_per_ds_variant = defaultdict(defaultdict)
for di, dataset in enumerate(datasets):
  dic_per_ds_variant[dataset] = defaultdict(float)
  for vi, variant in enumerate(variants):
    for mi, method in enumerate(method_names):
      # theta=10, 50の場合のカウントを足す(dataset, variantごとに)
      offs = 16 * vi
      dic_per_ds_variant[dataset][variant] += (np.count_nonzero(stat_res_arr[mi][di+offs]) + \
                                                np.count_nonzero(stat_res_arr[mi][di+8+offs])) / 60

In [18]:
dic_per_ds_variant

defaultdict(collections.defaultdict,
            {'3': defaultdict(float,
                         {'srnn': 0.8666666666666666,
                          'gru': 0.7833333333333333,
                          'lstm': 0.3666666666666667}),
             '4': defaultdict(float,
                         {'srnn': 0.4666666666666667,
                          'gru': 0.3666666666666667,
                          'lstm': 0.5666666666666667}),
             '7': defaultdict(float,
                         {'srnn': 0.3333333333333333,
                          'gru': 0.5666666666666667,
                          'lstm': 0.8666666666666667}),
             'bp': defaultdict(float,
                         {'srnn': 0.23333333333333334,
                          'gru': 0.3333333333333333,
                          'lstm': 0.2833333333333333}),
             'mr': defaultdict(float,
                         {'srnn': 0.7999999999999999,
                          'gru': 0.8166666666666667,
                

# method, variantごとに検定結果をまとめる
もうわけわかんねーや＾＾

In [23]:
datasets = ['3', '4', '7', 'bp', 'mr', 'imdb', 'toxic', 'mnist']
deno = len(thetas) * 6 * len(datasets)
dic_per_me_variant = defaultdict(defaultdict)

for mi, method in enumerate(method_names):
  dic_per_me_variant[method] = defaultdict(float)
  for vi, variant in enumerate(variants):
    for di, dataset in enumerate(datasets):
      # theta=10, 50の場合のカウントを足す(dataset, variantごとに)
      offs = 16 * vi
      dic_per_me_variant[method][variant] += (np.count_nonzero(stat_res_arr[mi][di+offs]) + \
                                                np.count_nonzero(stat_res_arr[mi][di+8+offs])) / deno

In [24]:
dic_per_me_variant

defaultdict(collections.defaultdict,
            {'ochiai': defaultdict(float,
                         {'srnn': 0.6354166666666666,
                          'gru': 0.6875,
                          'lstm': 0.625}),
             'tarantula': defaultdict(float,
                         {'srnn': 0.71875,
                          'gru': 0.6979166666666667,
                          'lstm': 0.6458333333333334}),
             'dstar': defaultdict(float,
                         {'srnn': 0.6145833333333333,
                          'gru': 0.65625,
                          'lstm': 0.5833333333333334}),
             'ochiai2': defaultdict(float,
                         {'srnn': 0.8020833333333334,
                          'gru': 0.7291666666666667,
                          'lstm': 0.6875}),
             'ample': defaultdict(float,
                         {'srnn': 0.6041666666666666,
                          'gru': 0.6770833333333334,
                          'lstm': 0.645833333333333

In [9]:
df_per_ds = pd.DataFrame.from_dict(dic_per_ds_variant)
display(df_per_ds)
np.savetxt('./stat_res_per_ds.csv', df_per_ds.values, delimiter=',', fmt='%f')

,3,4,7,bp,mr,imdb,toxic,mnist
srnn,0.866667,0.466667,0.333333,0.233333,0.800000,0.833333,0.866667,1.000000
gru,0.783333,0.366667,0.566667,0.333333,0.816667,0.750000,0.900000,1.000000
lstm,0.366667,0.566667,0.866667,0.283333,0.733333,0.533333,0.833333,0.916667


In [10]:
df_per_me = pd.DataFrame.from_dict(dic_per_me_variant)
display(df_per_me)
np.savetxt('./stat_res_per_method.csv', df_per_me.values, delimiter=',', fmt='%f')

,ochiai,tarantula,dstar,ochiai2,ample
srnn,0.635417,0.718750,0.614583,0.802083,0.604167
gru,0.687500,0.697917,0.656250,0.729167,0.677083
lstm,0.625000,0.645833,0.583333,0.687500,0.645833


mnistだけ効果量大で取り出せた場合 > 平均で勝った場合になってるのがまじでわからん
平均を計算し直してみる

In [14]:
for me in method_names:
  boot_list = np.zeros((10, 48, 6))
  for bi in range(10):
    pafl_path = f'./artifacts/pafl_res_fortest/boot_{bi}/{me}_pafl_res.csv'
    boot_list[bi, :, :] = np.genfromtxt(pafl_path, delimiter=',', dtype='float').astype(np.float32)
  boot_avg = np.nanmean(boot_list, axis=0)
  # 保存
  save_path = f'./artifacts/pafl_res_fortest/_boot_avg/{me}_pafl_res.csv'
  os.makedirs(os.path.dirname(save_path), exist_ok=True)
  np.savetxt(save_path, boot_avg, delimiter=',', fmt='%.3f')